# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from glob import glob
import os

from pyopia.classify import Classify
import pyopia.process
import pyopia.io
import pyopia.background
import exampledata
from pyopia.pipeline import Pipeline
import pyopia.instrument.holo as holo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
#download the default holo test data and create file list
foldername = exampledata.get_folder_from_holo_repository()
files = glob(os.path.join(foldername, '*.pgm'))

In [22]:
# example future pre-processing admin:
# 1) config file config load
# 2) build steps dict from loaded file
# 3) establish non-processing related metadata

os.makedirs('proc', exist_ok=True)

datafile_hdf = 'proc/holotest'
model_path = exampledata.get_example_model()
threshold = 0.9

bgfiles = files[:10] # provide a list of background files used to create the static background

holo_initial_settings = {'pixel_size': 4.4, # pixel size in um
                        'wavelength': 658, # laser wavelength in nm
                        'minZ': 22, # minimum reconstruction distance in mm
                        'maxZ': 60, # maximum reconstruction distance in mm
                        'stepZ': 2} #step size in mm

steps = {'initial': holo.Initial(files[0], **holo_initial_settings),
         'classifier': Classify(model_path=model_path),
         'create background': pyopia.background.CreateBackground(bgfiles,
                                                                 pyopia.instrument.holo.load_image),
         'load': holo.Load(),
         'correct background': pyopia.background.CorrectBackgroundAccurate(),
         'reconstruct': holo.Reconstruct(stack_clean=0),
         'segmentation': pyopia.process.Segment(threshold=threshold),
         'statextract': pyopia.process.CalculateStats(),
         'output': pyopia.io.StatsH5(datafile_hdf)}

processing_pipeline = Pipeline(steps)


Initialising pipeline
  Running <pyopia.instrument.holo.Initial object at 0x7fd344faabb0>
Using first raw file to determine image dimensions
Build kernel
HoloInitial done 2023-01-11 16:00:21.833172
  Running <pyopia.classify.Classify object at 0x7fd344faae80>
Pipeline ready with these data:  ['kern', 'cl']


In [24]:

for filename in files:
    stats = processing_pipeline.run(filename)

calling:  <class 'pyopia.background.CreateBackground'>  with:  ['kern', 'cl', 'filename', 'steps_string', 'bgstack', 'imbg', 'timestamp', 'img']
calling:  <class 'pyopia.instrument.holo.Load'>  with:  ['kern', 'cl', 'filename', 'steps_string', 'bgstack', 'imbg', 'timestamp', 'img']
calling:  <class 'pyopia.background.CorrectBackgroundAccurate'>  with:  ['kern', 'cl', 'filename', 'steps_string', 'bgstack', 'imbg', 'timestamp', 'img', 'imraw']


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:

# display metadata in the h5
pyopia.io.show_h5_meta(datafile_hdf + '-STATS.h5')

In [ ]:
# load the stats DataFrame from the h5 file
stats = pd.read_hdf(datafile_hdf + '-STATS.h5', 'ParticleStats/stats')
print('stats header: ', stats.columns)
print('Total number of particles: ', len(stats))